<a href="https://colab.research.google.com/github/lastplay-tyt/lastplay-tyt/blob/main/reservations_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#Importing the dataset
df = pd.read_csv('reservations.csv')

In [ ]:
#Check first 5 rows
df.head()

,start_date,end_date,friendly_date,duration,room_number
0,2024-01-06 14:30:00,2024-01-06 18:00:00,"Jan. 06, 2:30 - 6:00PM",210.0,109
1,2024-01-06 14:27:45,2024-01-06 16:27:45,"Jan. 06, 2:27 - 4:27PM",120.0,033
2,2024-01-06 15:22:29,2024-01-06 16:22:29,"Jan. 06, 3:22 - 4:22PM",60.0,326
3,2024-01-06 15:37:48,2024-01-06 17:37:48,"Jan. 06, 3:37 - 5:37PM",120.0,107
4,2024-01-06 15:54:41,2024-01-06 17:54:41,"Jan. 06, 3:54 - 5:54PM",120.0,032


In [ ]:
#Check data types of columns 'start_date' and 'end_date'
df['start_date'].dtypes

dtype('O')

In [ ]:
#like we saw from the EDA, the start and the end date are of a type obj

df['end_date'] = pd.to_datetime(df['end_date'])
df['start_date'] = pd.to_datetime(df['start_date'])

## We need to check whether the duration matches our start and end time

In [ ]:
#Calculating the minutes for each booking
df['calculated_duration'] = ((df['end_date'] - df['start_date']).dt.total_seconds() / 60)
duration_mismatch = df[df['duration'] != df['calculated_duration']]

duration_mismatch.head()


,start_date,end_date,friendly_date,duration,room_number,calculated_duration
106,2024-01-08 11:40:00,2024-01-08 12:52:00,"Jan. 08, 11:40AM - 12:52PM",70.000000,034,72.000000
120,2024-01-08 09:15:00,2024-01-08 10:33:00,"Jan. 08, 9:15 - 10:33AM",80.000000,104,78.000000
121,2024-01-08 08:50:31,2024-01-08 09:59:00,"Jan. 08, 8:50 - 9:59AM",70.483333,012,68.483333
122,2024-01-08 09:00:18,2024-01-08 09:59:00,"Jan. 08, 9:00 - 9:59AM",60.700000,108,58.700000
127,2024-01-08 09:17:48,2024-01-08 10:19:48,"Jan. 08, 9:17 - 10:19AM",60.000000,324,62.000000


# Replace duration with calculated_duration

In [ ]:
#remove the friendly duration column

df.drop(columns=['duration'], inplace=True)

In [ ]:
#remove the friendly duration column

df.drop(columns=['friendly_date'], inplace=True)

In [ ]:
df.head()

,start_date,end_date,friendly_date,room_number,calculated_duration
0,2024-01-06 14:30:00,2024-01-06 18:00:00,"Jan. 06, 2:30 - 6:00PM",109,210.0
1,2024-01-06 14:27:45,2024-01-06 16:27:45,"Jan. 06, 2:27 - 4:27PM",033,120.0
2,2024-01-06 15:22:29,2024-01-06 16:22:29,"Jan. 06, 3:22 - 4:22PM",326,60.0
3,2024-01-06 15:37:48,2024-01-06 17:37:48,"Jan. 06, 3:37 - 5:37PM",107,120.0
4,2024-01-06 15:54:41,2024-01-06 17:54:41,"Jan. 06, 3:54 - 5:54PM",032,120.0


In [ ]:
#rename 'calculated duration' to 'duration'

df = df.rename(columns={"calculated_duration":"duration"})

In [ ]:
df.head()

,start_date,end_date,room_number,duration
0,2024-01-06 14:30:00,2024-01-06 18:00:00,109,210.0
1,2024-01-06 14:27:45,2024-01-06 16:27:45,033,120.0
2,2024-01-06 15:22:29,2024-01-06 16:22:29,326,60.0
3,2024-01-06 15:37:48,2024-01-06 17:37:48,107,120.0
4,2024-01-06 15:54:41,2024-01-06 17:54:41,032,120.0


In [ ]:
#average duration per booking

avg_duration = df['duration'].mean()

avg_duration

np.float64(113.47853522684883)

In [ ]:
#round the duration column

df['duration'] = df['duration'].round()

## Check if any bookings > 9 hours


In [ ]:
long_booking = df[df['duration']>660]

print(len(long_booking))


258


In [ ]:
drop_rows = []

for index, row in df.iterrows():
  if row['duration'] > 660:
    drop_rows.append(index)

df.drop(index=drop_rows, inplace=True)

## Remove rows containing Saturday and Sunday

In [ ]:
drop_rows = []

for index, row in df.iterrows():
  start_day = row['start_date'].weekday()
  end_day = row['end_date'].weekday()
  if start_day in [5,6] or end_day in [5,6]:
    drop_rows.append(index)

df.drop(index=drop_rows, inplace=True)


In [ ]:
#Check if any remaining start_date is Saturday or Sunday
has_weekend_starts = df['start_date'].dt.weekday.isin([5, 6]).any()

has_weekend_ends = df['end_date'].dt.weekday.isin([5, 6]).any()

print("Any Saturday/Sunday in start_date:", has_weekend_starts)
print("Any Saturday/Sunday in end_date:", has_weekend_ends)

Any Saturday/Sunday in start_date: False
Any Saturday/Sunday in end_date: False


**Adding a synthetic dataset**

In [ ]:
df.head()

,start_date,end_date,room_number,duration
44,2024-01-08 15:30:00,2024-01-08 16:30:00,227F,60.0
45,2024-01-08 15:00:00,2024-01-08 16:15:00,103,75.0
46,2024-01-08 14:00:00,2024-01-08 15:15:00,141,75.0
47,2024-01-08 09:00:00,2024-01-08 09:10:00,210,10.0
48,2024-01-08 14:15:00,2024-01-08 16:15:00,210,120.0


# Capacity for each room

**The next block of code contains the functions which generates values for the number of attendees for each booking meeting.**

In [ ]:
def generateSyntheticDataset_nonGaussian(time_spent_vector, scaling_factor, no_of_rows):

    #Creating a random number generator
    rng = np.random.default_rng()

    #Creating a vector of random errors
    distribution_of_errors = rng.normal(0, 1, no_of_rows)

    #We are going to assume a relationship where time spent in a meeting is proportional to number of attendees.
    number_of_individuals = [time_spent_vector.iloc[i] *(1-np.exp(-scaling_factor)) + distribution_of_errors[i] for i in range(0, no_of_rows)]

    #if sta
    number_of_individuals = [int(number_of_individuals[i]) for i in range(len(number_of_individuals))]

    synthetic_data = pd.DataFrame({'Number of Individuals':number_of_individuals})

    return synthetic_data

In [ ]:
duration_vector = df['duration']

In [ ]:
generateSyntheticDataset_nonGaussian(duration_vector, 0.2, len(duration_vector))

,Number of Individuals
0,11
1,14
2,14
3,2
4,21
...,...
42279,21
42280,10
42281,2
42282,2
